In [23]:
from keras.applications.resnet50 import ResNet50
from keras.layers import *
from keras.preprocessing import image
from keras.optimizers import Adam
from keras.models import Model
from keras.utils import np_utils
import os
import random
import numpy as np
import matplotlib.pyplot as plt

In [2]:
data_path = "/home/azhad56/Desktop/Python_noida/datasets/Images/"

In [4]:
folders = os.listdir(data_path)

In [6]:
print(folders)

['cats', 'dogs', 'horses', 'humans']


In [7]:
image_data = []
image_labels = []
image_dict = {
    "cats":0,
    "dogs":1,
    "horses":2,
    "humans":3
}

In [12]:
for f in folders:
    sub_path = os.path.join(data_path,f)
    for img in os.listdir(sub_path):
        img_path = os.path.join(sub_path,img)
        img = image.load_img(img_path,target_size=((224,224)))
        img_arr = image.img_to_array(img)
        image_data.append(img_arr)
        image_labels.append(image_dict[f])

In [13]:
print(len(image_data),len(image_labels))

808 808


In [15]:
image_labels[:5]

[0, 0, 0, 0, 0]

In [18]:
combined = list(zip(image_data,image_labels))
random.shuffle(combined)
image_data[:],image_labels[:] = zip(*combined)

In [20]:
print(image_labels[:15])

[3, 2, 3, 3, 3, 0, 0, 0, 3, 3, 0, 2, 3, 1, 3]


In [21]:
X_train = np.array(image_data)
y_train = np_utils.to_categorical(np.array(image_labels))

In [22]:
print(X_train.shape,y_train.shape)

(808, 224, 224, 3) (808, 4)


In [26]:
base_model = ResNet50(include_top=False,weights='imagenet',input_shape=(224,224,3))

94773248/94765736 [==============================] - 61s 1us/step


In [27]:
base_model.summary()

Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_2[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
___________________________________________________________________________________________

In [28]:
gap = GlobalAveragePooling2D()(base_model.output)
fc1 = Dense(256,activation='relu')(gap)
d1 = Dropout(0.5)(fc1)
fc2 = Dense(4,activation='softmax')(d1)


In [29]:
model = Model(inputs = base_model.input,outputs = fc2)

In [30]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_2[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
______________________________________________________________________________________________

In [31]:
adam = Adam(learning_rate=0.00003)
model.compile(optimizer=adam,loss='categorical_crossentropy',metrics = ['accuracy'])

In [33]:
for ix in range(len(model.layers)):
    print(ix,model.layers[ix])

0 <tensorflow.python.keras.engine.input_layer.InputLayer object at 0x7fbebdfbdb90>
1 <tensorflow.python.keras.layers.convolutional.ZeroPadding2D object at 0x7fbebdfbda90>
2 <tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7fbebdfbde90>
3 <tensorflow.python.keras.layers.normalization_v2.BatchNormalization object at 0x7fbebdfc3790>
4 <tensorflow.python.keras.layers.core.Activation object at 0x7fbebdfc2b10>
5 <tensorflow.python.keras.layers.convolutional.ZeroPadding2D object at 0x7fbebdfc9810>
6 <tensorflow.python.keras.layers.pooling.MaxPooling2D object at 0x7fbebdfe4750>
7 <tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7fbebdff3650>
8 <tensorflow.python.keras.layers.normalization_v2.BatchNormalization object at 0x7fbebdf99f50>
9 <tensorflow.python.keras.layers.core.Activation object at 0x7fbebdfa1a90>
10 <tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7fbebdfa65d0>
11 <tensorflow.python.keras.layers.normalization_v2.BatchNormalization o

In [34]:
for ix in range(69):
    model.layers[ix].trainable = False

In [35]:
model.compile(optimizer=adam,loss='categorical_crossentropy',metrics = ['accuracy'])

In [36]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_2[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
______________________________________________________________________________________________

In [37]:
hist = model.fit(X_train,y_train,batch_size=16,shuffle=True,epochs=10,validation_split=0.20)

Epoch 1/10


KeyboardInterrupt: 

In [ ]:
h = hist.history

In [ ]:
plt.style.use('seaborn')
plt.plot(h['loss'],label='Training Loss')
plt.plot(h['val_loss'],label='Validation Loss')
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend()
plt.show()

In [ ]:
plt.style.use('seaborn')
plt.plot(h['acc'],label='Training acc')
plt.plot(h['val_acc'],label='Validation acc')
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.legend()
plt.show()